# Antidistinguishability conjecture

In this notebook, we will investigate the antidistinguishability conjecture and provide an explicit counterexample.

## Imports

In [1]:
import cvxopt
import numpy as np
import picos

Generate a random collection of $d$ (normalized) $d$-dimensional vectors. 

In [2]:
def generate_random_vectors(dim: int) -> list[np.ndarray]:
    """Return a list of "d" d-dimensional Haar-random vectors.

    Returns:
        An np.ndarray of "d" d-dimensional complex random vectors.
    """
    components = np.random.randn(dim * dim, 2).view(np.complex128)
    vectors: list[np.ndarray] = []
    for v in range(dim):
        vectors.append([])
        for c in range(dim):
            vectors[v].append(components[v * dim + c][0])

        # Normalize vectors.
        vectors[v] = vectors[v] / np.linalg.norm(vectors[v])
    return np.array(vectors)

Implements the following dual problem to obtain an upper bound on the antidistinguishability:

$$
    \begin{equation}
        \begin{aligned}
            \textbf{Dual:} \quad & \\
            \text{maximize:} \quad & \text{Tr}(H) \\
            \text{subject to:} \quad & Y \leq |\psi_i \rangle \langle \psi_i| \quad (\text{for each} \ i = 1, \ldots, d), \\
                                     & Y \in \text{Herm}(\mathcal{Y}).
        \end{aligned}
    \end{equation}
$$

In [3]:
def dual_problem(vectors: np.ndarray) -> float:
    problem = picos.Problem()

    # Convert vectors into corresponding density matrices.
    state_mtx = cvxopt.matrix(vectors)
    num_states, dim = state_mtx.size[0], state_mtx.size[1]    
    
    density_matrices: list[picos.Constant] = []
    for i in range(len(vectors)):
        mtx = state_mtx[i, :].H * state_mtx[i, :]        
        density_matrices.append(picos.Constant("ρ[{}]".format(i), mtx))
    
    # Set up variables and constraints for SDP:
    y_var = picos.HermitianVariable("Y", (dim, dim))
    problem.add_list_of_constraints([y_var << p for p in density_matrices])

    # Objective function:
    problem.set_objective("max", picos.trace(y_var))
    solution = problem.solve()

    return solution.value

Recall that an ensemble of states are *antidistinguishable* if and only if the optimal value of the SDP is equal to zero. Otherwise, if it is greater than zero, it is *not* antidistinguishable.

In [4]:
def is_antidistinguishable(vectors: np.ndarray) -> bool:
    """A value > 0 indicates the states are *not* antidistinguishable."""
    return True if np.isclose(dual_problem(vectors), 0) else False

Check if the conjectured inequality

$$
|\langle \psi_i | \psi_j \rangle| \leq \frac{d-2}{d-1}
$$

for all $i \not= j$ is satisfied.

In [5]:
def is_inequality_satisfied(vectors: np.ndarray) -> bool:
    """The anti-distinguishability conjecture states that for |Ψ_1>, ..., |Ψ_d> pure states, 
    if |<Ψ_i|Ψ_j>| ≤ (d − 2)/(d − 1) for all i != j, then the states are antidistinguishable.

    Returns:
        True if the inequality is satisfied, False otherwise.
    """
    dims = vectors[0].shape
    for i, _ in enumerate(vectors):
        for j, _ in enumerate(vectors):
            if i != j:
                ip_val = np.abs(np.trace(vectors[i].conj().T * vectors[j].reshape(-1, 1)))

                # If |<Ψ_i|Ψ_j>| > (d − 2)/(d − 1) for some i != j, return False.
                if ip_val > (dim - 2)/(dim - 1):
                    return False
    return True

Recall that a valid counterexample is one where the state of states are **not** antidistinguishable but **do** satisfy the conjectured bound.

In [6]:
def is_conjecture_violated(vectors: np.ndarray) -> bool:
    """Check if the antidistinguishability conjecture is satisfied.

    Returns:
        True if the conjecture is violated, False otherwise.
    """
    # The conjecture states that if the inequality is satisfied, then the states are antidistinguishable. 
    # If we find a violation of this fact then we have a violation of the conjecture.
    return is_inequality_satisfied(vectors) and not is_antidistinguishable(vectors)

## A counterexample to the antidistinguishability conjecture

One such counterexample that was found through random search is the following:
$$
\begin{equation}
    \begin{aligned} 
        |\psi_1\rangle = \begin{bmatrix} 
             +0.50127198 - 0.037607i \\ 
            -0.00698152 - 0.590973i \\ 
             +0.08186514 - 0.4497548i \\
            -0.01299883 + 0.43458491i 
        \end{bmatrix}, &\quad
        |\psi_2\rangle = \begin{bmatrix} 
            -0.07115345 - 0.27080326i \\ 
             +0.82047712 + 0.26320823i \\ 
             +0.22105089 - 0.2091996i \\ 
            -0.23575591 - 0.1758769i       
        \end{bmatrix}, \\ 
        |\psi_3\rangle = \begin{bmatrix} 
            +0.31360906 + 0.46339313i \\ 
           -0.0465825 - 0.47825017i \\ 
           -0.10470394 - 0.11776404i \\ 
            +0.60231515 + 0.26154959i 
        \end{bmatrix}, &\quad
        |\psi_4\rangle = \begin{bmatrix} 
            -0.53532122 - 0.03654632i \\
             +0.40955941 - 0.15150576i \\ 
            -0.05741386 + 0.23873985i \\
            -0.4737113 - 0.48652564i \\ 
        \end{bmatrix}.
    \end{aligned}
\end{equation} 
$$

We can observe that the above set of $4$ states violates the conjecture.

In [7]:
dim: int = 4
vectors = np.array(
    [
        [
            0.50127198 - 0.037607j,
            -0.00698152 - 0.590973j,
            0.08186514 - 0.4497548j,
            -0.01299883 + 0.43458491j,
        ],
        [
            -0.07115345 - 0.27080326j,
            0.82047712 + 0.26320823j,
            0.22105089 - 0.2091996j,
            -0.23575591 - 0.1758769j,
        ],
        [
            0.31360906 + 0.46339313j,
            -0.0465825 - 0.47825017j,
            -0.10470394 - 0.11776404j,
            0.60231515 + 0.26154959j,
        ],
        [
            -0.53532122 - 0.03654632j,
            0.40955941 - 0.15150576j,
            -0.05741386 + 0.23873985j,
            -0.4737113 - 0.48652564j,
        ],
    ]
)

print(f"Are the states antidistinguishable: {is_antidistinguishable(vectors)}")
print(f"Is inequality satisfied: {is_inequality_satisfied(vectors)}")
print(f"Is conjecture violated: {is_conjecture_violated(vectors)}")

Are the states antidistinguishable: False
Is inequality satisfied: True
Is conjecture violated: True
